<a href="https://colab.research.google.com/github/Ankushprajapatiap/YBI_PROJECT/blob/main/ybi_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 0: Installing extras
!pip install -q numpy pandas scikit-learn matplotlib plotly ipywidgets pymc3 arviz
# Enabling widgets in Colab:
from google.colab import output
output.enable_custom_widget_manager()

# Cell 1
import numpy as np, pandas as pd
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing(as_frame=True)
df = data.frame
df.head()
df.describe()

# Cell 2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['MedHouseVal']).values
y = df['MedHouseVal'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

# Cell 3
# Adding bias column
Xb = np.hstack([np.ones((X_train_s.shape[0],1)), X_train_s])
Xt = X_test_s
Xt_b = np.hstack([np.ones((Xt.shape[0],1)), Xt])

# Normalizing equation
theta_closed = np.linalg.pinv(Xb.T @ Xb) @ Xb.T @ y_train

# Predictions
y_pred_closed = Xt_b @ theta_closed

# Evaluation
from sklearn.metrics import mean_squared_error, r2_score
print("Closed-form MSE:", mean_squared_error(y_test, y_pred_closed))
print("Closed-form R2:", r2_score(y_test, y_pred_closed))

# Cell 4: gradient descent
def gradient_descent(X, y, lr=0.01, n_iter=5000):
    m, n = X.shape
    theta = np.zeros(n)
    for i in range(n_iter):
        grad = (2/m) * X.T @ (X @ theta - y)
        theta -= lr * grad
    return theta

Xb_all = np.hstack([np.ones((X_train_s.shape[0],1)), X_train_s])
theta_gd = gradient_descent(Xb_all, y_train, lr=0.01, n_iter=5000)

# test preds
Xt_b = np.hstack([np.ones((X_test_s.shape[0],1)), Xt])
y_pred_gd = Xt_b @ theta_gd
print("GD MSE:", mean_squared_error(y_test, y_pred_gd), "R2:", r2_score(y_test, y_pred_gd))

# Cell 5
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train_s, y_train)
y_pred_sklearn = lr.predict(X_test_s)
print("sklearn MSE:", mean_squared_error(y_test, y_pred_sklearn))
print("sklearn R2:", r2_score(y_test, y_pred_sklearn))

# Cell 6
import plotly.graph_objs as go
idx = np.arange(len(y_test))
fig = go.Figure()
fig.add_trace(go.Scatter(x=idx, y=y_test, mode='markers', name='actual'))
fig.add_trace(go.Scatter(x=idx, y=y_pred_closed, mode='markers', name='pred_closed'))
# Adding y=x line for perfect predictions
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode='lines', name='perfect_prediction', line=dict(color='red', dash='dash')))
fig.update_layout(title='Actual vs Predicted (closed form)', xaxis_title='Actual Values', yaxis_title='Predicted Values')
fig.show()

# Correlation Heatmap
import matplotlib.pyplot as plt
import seaborn as sns

corr = df.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=False, cmap='coolwarm')
plt.title("Feature Correlation Heatmap - California Housing Dataset")
plt.show()

# Installing latest PyMC
!pip install pymc arviz --quiet

import pymc as pm
import arviz as az
import numpy as np

# Using only first 3 features for faster sampling
X_small = X_train_s[:, :3]
X_small_test = X_test_s[:, :3]

with pm.Model() as linear_model:
    # Priors for parameters
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    betas = pm.Normal('betas', mu=0, sigma=1, shape=X_small.shape[1])
    sigma = pm.HalfNormal('sigma', sigma=1)

    # Expected value
    mu = alpha + pm.math.dot(X_small, betas)

    # Likelihood
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y_train)

    # Sampling posterior
    trace = pm.sample(1000, tune=1000, target_accept=0.95, random_seed=42)

#  Extracting parameter samples from posterior
alpha_samp = trace.posterior['alpha'].stack(draws=("chain","draw")).values  # (draws,)
betas_samp = trace.posterior['betas'].stack(draws=("chain","draw")).values  # (features, draws)

# Predictions for test set
preds = alpha_samp + X_small_test @ betas_samp   # (n_test, draws)

# Predictive mean & 95% credible intervals
pred_mean = preds.mean(axis=1)  # (n_test,)
pred_lower = np.percentile(preds, 2.5, axis=1)
pred_upper = np.percentile(preds, 97.5, axis=1)

# Metrics
from sklearn.metrics import mean_squared_error, r2_score
print("Bayesian pred mean MSE:", mean_squared_error(y_test, pred_mean))
print("Bayesian pred mean R2:", r2_score(y_test, pred_mean))

# Plot predictions with uncertainty for first 50 samples
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.plot(pred_mean[:50], label='Predicted Mean')
plt.fill_between(np.arange(50), pred_lower[:50], pred_upper[:50], alpha=0.3, label='95% Credible Interval')
plt.plot(y_test[:50], label='Actual', marker='o', linestyle='')
plt.legend()
plt.title('Bayesian Linear Regression with Predictive Intervals')
plt.show()

# Cell 8 - counterfactual for closed-form linear model
import scipy.optimize as opt

# picking a test instance
i = 10
x0 = X_test_s[i].copy()
y0 = y_test[i]
target = y0 + 1.0  # want to increase prediction by 1

# objective: minimize ||delta||^2 s.t. model(x0+delta) = target
# for linear model model(x) = w0 + w^T x  (useing closed form theta_closed)
w0 = theta_closed[0]
w = theta_closed[1:]

def objective(delta):
    return np.sum(delta**2)

def constraint(delta):
    return (w0 + w.dot(x0 + delta)) - target

cons = ({'type':'eq','fun':constraint})
res = opt.minimize(objective, np.zeros_like(x0), constraints=cons)
delta = res.x
new_x = x0 + delta
print("Smallest L2 change norm:", np.linalg.norm(delta))
print("Original pred:", (w0 + w.dot(x0)), "New pred:", (w0 + w.dot(new_x)))

# Installing ipywidgets
!pip install ipywidgets --quiet
from IPython.display import display, clear_output
import ipywidgets as widgets
import numpy as np

# Extracting Bayesian posterior samples
alpha_samp = trace.posterior['alpha'].stack(draws=("chain","draw")).values
betas_samp = trace.posterior['betas'].stack(draws=("chain","draw")).values  # shape: (3 features, draws)

# Feature names exactly as in Bayesian model training
feature_names = ['MedInc', 'HouseAge', 'AveRooms']


try:
    base_features = X_small_test[0].copy()
except NameError:
    base_features = np.mean(X_train_s[:, :3], axis=0)  # fallback to mean values

# Creating sliders for each feature
sliders = []
for i, feat in enumerate(feature_names):
    min_val = float(np.min(X_train_s[:, i]))
    max_val = float(np.max(X_train_s[:, i]))
    sliders.append(
        widgets.FloatSlider(
            value=float(base_features[i]),
            min=min_val,
            max=max_val,
            step=0.1,
            description=feat
        )
    )

# Updating prediction
def update_prediction(*args):
    new_features = np.array([s.value for s in sliders])
    preds = alpha_samp + betas_samp.T @ new_features
    pred_mean = preds.mean()
    pred_lower = np.percentile(preds, 2.5)
    pred_upper = np.percentile(preds, 97.5)

    clear_output(wait=True)
    display(*sliders)
    print(f" Predicted Median House Value: {pred_mean:.2f}")
    print(f" 95% Credible Interval: [{pred_lower:.2f}, {pred_upper:.2f}]")

# Linking sliders
for s in sliders:
    s.observe(update_prediction, 'value')

# Showing initial UI
display(*sliders)
update_prediction()

FloatSlider(value=0.3245615565005424, description='MedInc', max=5.839267946533965, min=-1.7754384434994577)

FloatSlider(value=-0.9907655106085542, description='HouseAge', max=1.8561733525483044, min=-2.1907655106085544…

FloatSlider(value=27.59561378175808, description='AveRooms', max=57.16655150785949, min=-1.904386218241921)

 Predicted Median House Value: 0.28
 95% Credible Interval: [-0.06, 0.63]
